# Distributed HPO with Ray Tune and XGBoost-Ray

This demo introduces **Ray tune's** key concepts using a a classification example. This example is derived from [Hyperparametere Tuning with Ray Tune and XGBoost-Ray](https://github.com/ray-project/xgboost_ray#hyperparameter-tuning). Basically, there are three basic steps or Ray Tune pattern for you as a newcomer to get started with using Ray Tune.

 1. Setup your config space and define your trainable and objective function
 2. Use tune to execute your training, supplying the appropriate arguments including: search space, [search algorithms](https://docs.ray.io/en/latest/tune/api_docs/suggestion.html#blendsearch) or [trial schedulers](https://docs.ray.io/en/latest/tune/api_docs/schedulers.html#tune-schedulers)
 3. Examine analyse the results
 
 <img src="https://docs.ray.io/en/latest/_images/tune-workflow.png" height="50%" width="60%">


See also the [Hyperparameter Tuning References](References-Hyperparameter-Tuning.ipynb) notebook and the [Tune documentation](http://tune.io), in particular, the [API reference](https://docs.ray.io/en/latest/tune/api_docs/overview.html). 


In [1]:
from xgboost_ray import RayDMatrix, RayParams, train
from sklearn.datasets import load_breast_cancer

import ray
from ray import tune
CONNECT_TO_ANYSCALE=True

In [2]:
if ray.is_initialized:
    ray.shutdown()
    if CONNECT_TO_ANYSCALE:
        ray.init("anyscale://jsd-weekly-demo")
    else:
        ray.init()

Authenticating
Loaded Anyscale authentication token from ANYSCALE_CLI_TOKEN.

Output
(anyscale +0.7s) WARNING: No working_dir specified! Files will only be uploaded to the cluster if a working_dir is provided or a project is detected. In the future, files will only be uploaded if working_dir is provided. To ensure files continue being imported going forward, set the working_dir in your runtime environment. See https://docs.ray.io/en/latest/handling-dependencies.html#runtime-environments.
(anyscale +0.7s) .anyscale.yaml found in project_dir. Directory is attached to a project.
(anyscale +0.8s) Using project (name: prj-weekly-demo, project_dir: /Users/jules/git-repos/ray-core-tutorial, id: prj_5rvR1w2ciyUs9RM27FeZ6FVB).
(anyscale +3.0s) cluster jsd-weekly-demo is currently running, the cluster will not be restarted.


2022-01-09 16:03:10,694	INFO packaging.py:352 -- Creating a file package for local directory '/Users/jules/git-repos/ray-core-tutorial'.
2022-01-09 16:03:10,730	INFO packaging.py:221 -- Pushing file package 'gcs://_ray_pkg_58a639b07024fb1a.zip' (6.25MiB) to Ray cluster...
2022-01-09 16:03:16,033	INFO packaging.py:224 -- Successfully pushed file package 'gcs://_ray_pkg_58a639b07024fb1a.zip'.


(anyscale +20.4s) Connected to jsd-weekly-demo, see: https://console.anyscale.com/projects/prj_5rvR1w2ciyUs9RM27FeZ6FVB/clusters/ses_GKnU1MGrXqZNmAkicHxSKU9G
(anyscale +20.4s) URL for head node of cluster: https://session-gknu1mgrxqznmakichxsku9g.i.anyscaleuserdata.com


## Step 1: Define a 'Trainable' training function to use with Ray Tune `ray.tune(...)`

In [3]:
NUM_OF_ACTORS = 4           # degree of parallel trials; each actor will have a separate trial
NUM_OF_CPUS_PER_ACTOR = 1   # number of CPUs per actor

ray_params = RayParams(num_actors=NUM_OF_ACTORS, cpus_per_actor=NUM_OF_CPUS_PER_ACTOR)

In [4]:
def train_func_model(config:dict, checkpoint_dir=None):
    # create the dataset
    train_X, train_y = load_breast_cancer(return_X_y=True)
    # Convert to RayDMatrix data structure
    train_set = RayDMatrix(train_X, train_y)

    # Empty dictionary for the evaluation results reported back
    # to tune
    evals_result = {}

    # Train the model with XGBoost train
    bst = train(
        params=config,                       # our hyperparameter search space
        dtrain=train_set,                    # our RayDMatrix data structure
        evals_result=evals_result,           # place holder for results
        evals=[(train_set, "train")],
        verbose_eval=False,
        ray_params=ray_params)                # distributed parameters configs for Ray Tune

    bst.save_model("model.xgb")

## Step 2: Define a hyperparameter search space

In [5]:
 # Specify the typical hyperparameter search space
config = {
    "tree_method": "approx",
    "objective": "binary:logistic",
    "eval_metric": ["logloss", "error"],
    "eta": tune.loguniform(1e-4, 1e-1),
    "subsample": tune.uniform(0.5, 1.0),
    "max_depth": tune.randint(1, 9)
}

## Step 3: Run Ray tune main trainer and examine the results

Ray Tune will launch distributed HPO, using four remote actors, each with its own instance of the trainable func

<img src="images/ray_tune_dist_hpo.png" height="60%" width="70%"> 

In [6]:
# Run tune
analysis = tune.run(
    train_func_model,
    config=config,
    metric="train-error",
    mode="min",
    num_samples=4,
    verbose=1,
    resources_per_trial=ray_params.get_tune_resources()
)

(run pid=6439) == Status ==
(run pid=6439) Current time: 2022-01-09 16:03:23 (running for 00:00:00.12)
(run pid=6439) Memory usage on this node: 2.8/62.0 GiB
(run pid=6439) Using FIFO scheduling algorithm.
(run pid=6439) Resources requested: 0/80 CPUs, 0/0 GPUs, 0.0/216.19 GiB heap, 0.0/92.35 GiB objects
(run pid=6439) Result logdir: /home/ray/ray_results/train_func_model_2022-01-09_16-03-22
(run pid=6439) Number of trials: 4/4 (4 PENDING)
(run pid=6439) 
(run pid=6439) 


(ImplicitFunc pid=6511) 2022-01-09 16:03:25,577	INFO main.py:926 -- [RayXGBoost] Created 4 new actors (4 total actors). Waiting until actors are ready for training.
(ImplicitFunc pid=682, ip=172.31.53.244) 2022-01-09 16:03:25,677	INFO main.py:926 -- [RayXGBoost] Created 4 new actors (4 total actors). Waiting until actors are ready for training.
(ImplicitFunc pid=878, ip=172.31.52.146) 2022-01-09 16:03:25,707	INFO main.py:926 -- [RayXGBoost] Created 4 new actors (4 total actors). Waiting until actors are ready for training.
(ImplicitFunc pid=721, ip=172.31.39.40) 2022-01-09 16:03:25,718	INFO main.py:926 -- [RayXGBoost] Created 4 new actors (4 total actors). Waiting until actors are ready for training.
(ImplicitFunc pid=682, ip=172.31.53.244) 2022-01-09 16:03:27,794	INFO main.py:971 -- [RayXGBoost] Starting XGBoost training.
(_RemoteRayXGBoostActor pid=717, ip=172.31.53.244) [16:03:27] task [xgboost.ray]:140538438038000 got new rank 3
(_RemoteRayXGBoostActor pid=714, ip=172.31.53.244) [1

(run pid=6439) 2022-01-09 16:03:28,707	WARN commands.py:269 -- Loaded cached provider configuration
(run pid=6439) 2022-01-09 16:03:28,707	WARN commands.py:270 -- If you experience issues with the cloud provider, try re-running the command with --no-config-cache.
(run pid=6439) Authenticating
(run pid=6439) Loaded Anyscale authentication token from variable.
(run pid=6439) 
(run pid=6439) 2022-01-09 16:03:30,007	INFO command_runner.py:357 -- Fetched IP: 172.31.53.244
(run pid=6439) 2022-01-09 16:03:30,007	INFO log_timer.py:25 -- NodeUpdater: ins_wmBDm5Bam9yYSzTgTiYpHstj: Got IP  [LogTimer=65ms]
(run pid=6439) == Status ==
(run pid=6439) Current time: 2022-01-09 16:03:30 (running for 00:00:07.59)
(run pid=6439) Memory usage on this node: 3.5/62.0 GiB
(run pid=6439) Using FIFO scheduling algorithm.
(run pid=6439) Resources requested: 20.0/80 CPUs, 0/0 GPUs, 0.0/216.19 GiB heap, 0.0/92.35 GiB objects
(run pid=6439) Current best trial: b98f6_00002 with train-error=0.080844 and parameters={

(ImplicitFunc pid=721, ip=172.31.39.40) 2022-01-09 16:03:34,268	INFO main.py:1450 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 8.60 seconds (6.33 pure XGBoost training time).
(ImplicitFunc pid=682, ip=172.31.53.244) 2022-01-09 16:03:34,263	INFO main.py:1450 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 8.63 seconds (6.46 pure XGBoost training time).
(ImplicitFunc pid=878, ip=172.31.52.146) 2022-01-09 16:03:34,262	INFO main.py:1450 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 8.60 seconds (6.22 pure XGBoost training time).
(ImplicitFunc pid=6511) 2022-01-09 16:03:34,278	INFO main.py:1450 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 8.74 seconds (6.48 pure XGBoost training time).
(run pid=6439) 2022-01-09 16:03:34,403	INFO tune.py:626 -- Total run time: 12.09 seconds (11.12 seconds for the tuning loop).


(run pid=6439) == Status ==
(run pid=6439) Current time: 2022-01-09 16:03:34 (running for 00:00:11.13)
(run pid=6439) Memory usage on this node: 3.3/62.0 GiB
(run pid=6439) Using FIFO scheduling algorithm.
(run pid=6439) Resources requested: 0/80 CPUs, 0/0 GPUs, 0.0/216.19 GiB heap, 0.0/92.35 GiB objects
(run pid=6439) Current best trial: b98f6_00000 with train-error=0.017575 and parameters={'tree_method': 'approx', 'objective': 'binary:logistic', 'eval_metric': ['logloss', 'error'], 'eta': 0.00010981286584081387, 'subsample': 0.6900396002641832, 'max_depth': 5, 'nthread': 1, 'n_jobs': 1}
(run pid=6439) Result logdir: /home/ray/ray_results/train_func_model_2022-01-09_16-03-22
(run pid=6439) Number of trials: 4/4 (4 TERMINATED)
(run pid=6439) 
(run pid=6439) 


In [7]:
print("Best hyperparameters", analysis.best_config)

Best hyperparameters {'tree_method': 'approx', 'objective': 'binary:logistic', 'eval_metric': ['logloss', 'error'], 'eta': 0.00010981286584081387, 'subsample': 0.6900396002641832, 'max_depth': 5}


In [8]:
analysis.results_df.head(5)

/usr/local/anaconda3/envs/ray-core/lib/python3.8/site-packages/ray/tune/analysis/experiment_analysis.py:262: UserWarning: Dataframes will use '/' instead of '.' to delimit nested result keys in future versions of Ray. For forward compatibility, set the environment variable TUNE_RESULT_DELIM='/'
  warnings.warn(


,train-logloss,train-error,time_this_iter_s,done,timesteps_total,episodes_total,training_iteration,experiment_id,date,timestamp,...,iterations_since_restore,experiment_tag,config.tree_method,config.objective,config.eval_metric,config.eta,config.subsample,config.max_depth,config.nthread,config.n_jobs
trial_id,,,,,,,,,,,,,,,,,,,,,
b98f6_00000,0.692205,0.017575,0.006520,True,None,None,10,6c81cec979d14543b4a99a627da369df,2022-01-09_16-03-34,1641773014,...,10,"0_eta=0.00010981,max_depth=5,subsample=0.69004",approx,binary:logistic,"[logloss, error]",0.000110,0.690040,5,1,1
b98f6_00001,0.689884,0.019332,0.009151,True,None,None,10,606dc44a5fdf4952a5293e01cc9e116a,2022-01-09_16-03-34,1641773014,...,10,"1_eta=0.00038227,max_depth=8,subsample=0.69583",approx,binary:logistic,"[logloss, error]",0.000382,0.695830,8,1,1
b98f6_00002,0.608622,0.061511,0.005931,True,None,None,10,b6747593255b4bb7a170ac9c64ee8354,2022-01-09_16-03-34,1641773014,...,10,"2_eta=0.013487,max_depth=1,subsample=0.91302",approx,binary:logistic,"[logloss, error]",0.013487,0.913022,1,1,1
b98f6_00003,0.662892,0.019332,0.007406,True,None,None,10,f96e569ce8a040f8a527506288d8ae52,2022-01-09_16-03-34,1641773014,...,10,"3_eta=0.0036449,max_depth=5,subsample=0.6839",approx,binary:logistic,"[logloss, error]",0.003645,0.683903,5,1,1


---

In [9]:
ray.shutdown()

## References

 * [Ray Train: Tune: Scalable Hyperparameter Tuning](https://docs.ray.io/en/master/tune/index.html)
 * [Introducing Distributed XGBoost Training with Ray](https://www.anyscale.com/blog/distributed-xgboost-training-with-ray)
 * [How to Speed Up XGBoost Model Training](https://www.anyscale.com/blog/how-to-speed-up-xgboost-model-training)
 * [XGBoost-Ray Project](https://github.com/ray-project/xgboost_ray)
 * [Distributed XGBoost on Ray](https://docs.ray.io/en/latest/xgboost-ray.html)